# The pySIP library

The previous chapters on this series showed the theory behind linear stochastic state-space models. The theory was then applied by calibrating a simple 2R2C model on the heat dynamics of an unoccupied test house. The purpose was to give a transparent overview of the mechanics of state-space models and the Kalman filter. In practical applications however, we will wish to ensure that we obtained a robust predictive model, suitable for building energy performance assessment and prediction of energy use. Additional tools are required to assist in model selection, diagnostics of convergence, validation of calibrated models, and eventually Bayesian inference. This is why [the pySIP python library for stochastic state-space inference and prediction](https://github.com/locie/pySIP) was developed.

pySIP provides a framework for infering continuous time linear stochastic state-space models. For that purpose, it is possible to chose between a frequentist and a Bayesian workflow. Each workflow allows to estimate the parameters, assess the inference and model reliability, and perform model selection.

This page is an example of how pySIP can be used, with the same data and model structure as the previous notebook on state-space models. The following code is provided as an example in the pySIP repository.

We are considering the same experimental building, and the same measurement period, as in the [previous notebook](statespace_simple).

```{figure} /figures/rcexample_armadillo.jpg
---
name: rcexample_armadillo
width: 400px
align: center
---
Armadillo Box
```

The building is monitored by a variety of sensors, but the present study only uses records of indoor temperature and prescribed heating power, in addition to weather data. The indoor temperature profiles used here have been averaged over several sensors distributed in the living space. An experimental sequence of four days was used in this study.

Some imports first, then we load the [data](https://github.com/srouchier/buildingenergygeeks/blob/master/data/statespace.csv).


In [1]:
import pandas as pd
import numpy as np

from pysip.statespace import TwTi_RoRi
from pysip.regressors import Regressor

# Reading the data
df = pd.read_csv('data/statespace.csv').set_index('Time')
df.drop(df.index[-1], axis=0, inplace=True)

# pySIP's fit() method takes a dataframe df as argument
# It also needs to be passed the labels of inputs and outputs in this dataframe
inputs = ['To', 'Qh']
outputs = 'xi'

# The time can be scaled on daily units instead of seconds.
# This brings heat capacities to a range closer to other variables
sT = 3600.0 * 24.0
df.index /= sT

/home/simon/anaconda3/envs/bayes_env/lib/python3.11/site-packages/arviz/data/base.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources



We imported the `TwTi_RoRi` model, which is similar to the 2R2C model of the previous notebook, except that solar aperture coefficients are removed. This is fine since their estimated values were very low.

Parameters are specified as a list of dictionaries. Each parameter has a value in the constrained space $\theta$ and in the unconstrained space $\eta$. These two values are linked by the relation:

$$ \theta = \mathrm{loc} + f(\eta) . \mathrm{scale}$$

where `loc` and `scale` are keywords of a parameter's dictionary, indicating location and scaling values. A parameter may be transformed by a function $f$, in order to search on a log scale for instance. More options on parameter specification are given [in the documentation](https://locie.github.io/pySIP/core/parameter.html)

pySIP works with either a frequentist and a Bayesian regressor. The frequentist regressor, used in this example, applies maximum likelihood estimation by the BFGS algorithm.

In [2]:
# Specification of parameters
parameters = [
    dict(name='Ro', value=1.0, scale=0.1, transform='log'),
    dict(name='Ri', value=1.0, scale=0.01, transform='log'),
    dict(name='Cw', value=1.0, scale=1e7 / sT, transform='log'),
    dict(name='Ci', value=1.0, scale=1e6 / sT, transform='log'),
    dict(name='sigw_w', value=1.0, scale=0.01 * sT ** 0.5, transform='log'),
    dict(name='sigw_i', value=1.0, scale=0.01 * sT ** 0.5, transform='log'),
    dict(name='sigv', value=1.0, scale=0.01, transform='log'),
    dict(name='x0_w', value=1.0, scale=25.0, transform='log'),
    dict(name='x0_i', value=26.7),
    dict(name='sigx0_w', value=1.0, transform='fixed'),
    dict(name='sigx0_i', value=1.0, transform='fixed'),
]

# Definition of the regressor, and model fitting
reg = Regressor(TwTi_RoRi(parameters, hold_order=1))
out = reg.fit(df=df, inputs=inputs, outputs=outputs)

print(out[0])

/home/simon/anaconda3/envs/bayes_env/lib/python3.11/site-packages/pysip/regressors.py:349: OptimizeWarning: Unknown solver options: inputs, outputs
  results = minimize(


Optimization terminated successfully.
         Current function value: -179.300651
         Iterations: 43
         Function evaluations: 1007
         Gradient evaluations: 53
                 θ       σ(θ)        pvalue    |g(η)|     |dpen(θ)|
Ro        0.019375   0.001459  0.000000e+00  0.000031  2.663897e-15
Ri        0.002276   0.000098  0.000000e+00  0.000036  1.929695e-15
Cw      167.090370  10.039658  0.000000e+00  0.000027  4.798104e-17
Ci       30.264111   1.409343  0.000000e+00  0.000017  1.462570e-17
sigw_w    0.511984   0.064622  2.906564e-13  0.000008  3.296102e-15
sigw_i    0.000312   0.039940  9.937789e-01  0.000011  8.882998e-09
sigv      0.066973   0.003994  0.000000e+00  0.000027  2.229441e-18
x0_w     30.104148   1.054276  0.000000e+00  0.000044  6.896478e-17
x0_i     30.272107   0.934865  0.000000e+00  0.000001  0.000000e+00



Note that the parameter names are different than on the previous notebook, but the estimation results are similar. The calculation is much faster than what we did with scipy's `curve_fit` method, and also more reliable because of the choice of algorithm. 

Finally, we can compare the fitted model output with the data.

```{figure} /figures/rcexample05.png
---
name: rcexample05
width: 400px
align: center
---
```